# Hills
This notebook is focused on analyzing pusher performance.

1) Read a set of roll data
2) Read a name, buggy, start time data from a CSV
3) Split the roll data into segments
4) Associate names with each segment 
5) Compute statistics, average speed, number of pushes, average impulse?
6) Write statistics to a csv

In [ ]:
%matplotlib widget
import datetime
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from glob import glob
from ipywidgets import interact, BoundedIntText
from scipy.integrate import cumulative_simpson
from scipy.interpolate import splprep, splev

from constants import *
from plot_utils import *
from track import Track

In [ ]:
from track import Track
# Load track data and list all the processed files
file_patterns = [
    f'{DATA_DIR}/parsed/yyyy-mm-dd*',
]
parsed_files = [file for pattern in file_patterns for file in glob(pattern)]
reference_track=None
data = []
for file in parsed_files:
    label = os.path.splitext(os.path.basename(file))[0]
    data.append(Track(label, file))
    print(f'Loaded {label}')

In [ ]:
PUSHER = 'Pusher'
BUGGY = 'Buggy'
HILL = 'Hill'
START = 'Start Time'
DATE = 'Date'
columns = [PUSHER, BUGGY, HILL, START, DATE]

pusher_data = pd.read_csv(f'{DATA_DIR}/pushers/yyyy-mm-dd_pushers.csv')

if pd.api.types.is_numeric_dtype(pusher_data[HILL]):
    pusher_data[HILL] = pusher_data[HILL].map(lambda N: f'Hill {N}')
if not pd.api.types.is_datetime64_any_dtype(pusher_data[START]):
    dt = pusher_data[DATE] + 'T' + pusher_data[START]
    pusher_data[START] = pd.to_datetime(dt).dt.tz_localize('US/Eastern')
pusher_data

In [ ]:
GPS_START = 'GPS start time'
GPS_END = 'GPS end time'
DATA = 'DataFrame'
UNKNOWN = 'unknown'
columns = [BUGGY, HILL, GPS_START, GPS_END, START, PUSHER, DATA]
hills = []
for track in data:
    print(f'Scanning: {track.label}')
    buggy_name = track.buggy_name
    for hill_label, gps_start, gps_end in track.get_all_hill_splits():
        pusher = UNKNOWN
        df = track.time_slice(gps_start, gps_end)
        origin = track.date
        start = pd.to_datetime(gps_start % (24*60*60), unit='s', origin=origin).tz_localize('utc').tz_convert('US/Eastern')
        hills.append([buggy_name, hill_label, gps_start, gps_end, start, pusher, df])
hills = pd.DataFrame(hills, columns=columns)
# Convert GPS seconds of week to local time
# hills[START] = pd.to_datetime(hills[GPS_START] % (24*60*60), unit='s', origin=date).dt.tz_localize('utc').dt.tz_convert('US/Eastern')
hills

In [ ]:
for pusher, buggy, hill, start, date in pusher_data.to_numpy():
    mask = (hills[BUGGY] == buggy) & (hills[HILL] == hill)
    candidates = hills[mask]
    delta = (candidates[START] - start).abs()
    idx = delta.idxmin()
    prev = hills[PUSHER][idx]
    if prev != 'unknown' and prev != pusher:
        print(f'Conflict: {prev, pusher, buggy, hill, start, candidates[START][idx]}')
        continue
    hills.loc[idx, PUSHER] = pusher
print(f'Matched: {(hills[PUSHER] != UNKNOWN).sum()}')
print(f'Unmatched: {(hills[PUSHER] == UNKNOWN).sum()}')

In [ ]:
for label, group in hills.groupby(HILL):
    fig, ax = plt.subplots()
    fig.suptitle(label)
    ax.set_xlabel('Split Time (s)')
    ax.set_ylabel('Speed (m/s)')
    ax.set_ylim(0, 8)
    for idx in group.index:
        pusher = group[PUSHER][idx]
        df = group[DATA][idx]
        if label in [HILL_1, HILL_2]:
            align_to = df[df[Y] < HILLS[label].start.y].index[0]
        else:
            align_to = df[df[X] > HILLS[label].start.x].index[0]
        t = (df[T] - df[T][align_to]).to_numpy()
        s = df[SPEED].to_numpy()
        ax.plot(t, s, label=pusher)
        ax.axvline(x=0, color='gray', lw=0.8)
    ax.legend()
plt.show()